# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [121]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [122]:
city_data = "../output_data/cities.csv"
city_df = pd.read_csv(city_data)
city_df = city_df.drop(columns=["Unnamed: 0"])
city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Monrovia,80,LR,1595793344,78,6.30,-10.80,76.66,12.50
1,Bluff,0,NZ,1595792824,85,-46.60,168.33,42.24,11.50
2,Oddur,18,SO,1595793344,73,4.12,43.89,73.96,12.06
3,Ushuaia,40,AR,1595793009,90,-54.80,-68.30,33.80,20.80
4,Bathsheba,59,BB,1595793094,77,13.22,-59.52,83.12,17.09
...,...,...,...,...,...,...,...,...,...
546,Nuuk,20,GL,1595792922,55,64.18,-51.72,60.80,8.05
547,Khuzdar,3,PK,1595793471,52,27.74,66.64,82.58,4.74
548,Ferrol,0,ES,1595793471,77,43.48,-8.24,69.01,4.70
549,Kautokeino,20,NO,1595793471,86,69.01,23.04,55.18,8.63


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [123]:
gmaps.configure(api_key = g_key)

In [124]:
locations = city_df[["Lat", "Lng"]].astype(float)
humidity = city_df["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [125]:
below_80 = city_df.loc[city_df["Max Temp"] < 80]
ideal_temp = below_80.loc[city_df["Max Temp"] > 70]
temp_wind = ideal_temp.loc[ideal_temp["Wind Speed"] < 10]
weather_criteria_df = temp_wind.loc[temp_wind["Cloudiness"] == 0]
weather_criteria_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
258,Córdoba,0,AR,1595793165,16,-31.41,-64.18,75.99,8.05
322,High Prairie,0,CA,1595793203,52,55.43,-116.49,71.42,9.26
334,Yumen,0,CN,1595793422,22,40.28,97.20,71.64,5.37
337,Porto Torres,0,IT,1595793422,65,40.84,8.40,77.00,4.70
530,Mbanza-Ngungu,0,CD,1595792924,45,-5.25,14.87,75.67,7.47


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [126]:
hotel_df = pd.DataFrame(weather_criteria_df)
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
258,Córdoba,0,AR,1595793165,16,-31.41,-64.18,75.99,8.05
322,High Prairie,0,CA,1595793203,52,55.43,-116.49,71.42,9.26
334,Yumen,0,CN,1595793422,22,40.28,97.20,71.64,5.37
337,Porto Torres,0,IT,1595793422,65,40.84,8.40,77.00,4.70
530,Mbanza-Ngungu,0,CD,1595792924,45,-5.25,14.87,75.67,7.47


In [127]:
# add column for Hotel Name
hotel_df["Hotel Name"] = ""

# params
params = {"radius": 5000, "types": "lodging", "keyword": "Hotel", "key": g_key}

# use the lat & lng to identify hotels
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    # update location
    params["location"] = f"{lat},{lng}"

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run query and convert to json
    hotel_query = requests.get(base_url, params=params)
    hotel_query = hotel_query.json()

    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_query["results"][0]["name"]
    except:
        pass

hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
258,Córdoba,0,AR,1595793165,16,-31.41,-64.18,75.99,8.05,Azur Real Hotel Boutique
322,High Prairie,0,CA,1595793203,52,55.43,-116.49,71.42,9.26,Days Inn by Wyndham High Prairie
334,Yumen,0,CN,1595793422,22,40.28,97.20,71.64,5.37,
337,Porto Torres,0,IT,1595793422,65,40.84,8.40,77.00,4.70,Hotel Libyssonis
530,Mbanza-Ngungu,0,CD,1595792924,45,-5.25,14.87,75.67,7.47,Hotel Les Cascades
543,Guéret,0,FR,1595793469,64,46.17,1.87,73.00,4.70,Hôtel De Pommeil


In [128]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]

In [129]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

marker_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig


Figure(layout=FigureLayout(height='420px'))